# GraphCast

Edited graphcast_demo code which allows grid and mesh structures to be extracted. Also plots these.

This colab lets you run several versions of GraphCast.

The model weights, normalization statistics, and example inputs are available on [Google Cloud Bucket](https://console.cloud.google.com/storage/browser/dm_graphcast).

A Colab runtime with TPU/GPU acceleration will substantially speed up generating predictions and computing the loss/gradients. If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".

> <p><small><small>Copyright 2023 DeepMind Technologies Limited.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

# Installation and Initialization


In [ ]:
# @title Pip install graphcast and dependencies

%pip install --upgrade https://github.com/deepmind/graphcast/archive/master.zip

In [ ]:
# @title Workaround for cartopy crashes

# Workaround for cartopy crashes due to the shapely installed by default in
# google colab kernel (https://github.com/anitagraser/movingpandas/issues/81):
!pip uninstall -y shapely
!pip install shapely --no-binary shapely

In [ ]:
# @title Imports

import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray
import pickle
from google.colab import files, drive
import networkx as nx
import plotly.graph_objects as go

def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))


In [ ]:
# @title Authenticate with Google Cloud Storage

gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")
dir_prefix = "graphcast/"

# Load the Data and initialize the model

## Load the model params

Choose one of the two ways of getting model params:
- **random**: You'll get random predictions, but you can change the model architecture, which may run faster or fit on your device.
- **checkpoint**: You'll get sensible predictions, but are limited to the model architecture that it was trained with, which may not fit on your device. In particular generating gradients uses a lot of memory, so you'll need at least 25GB of ram (TPUv4 or A100).

Checkpoints vary across a few axes:
- The mesh size specifies the internal graph representation of the earth. Smaller meshes will run faster but will have worse outputs. The mesh size does not affect the number of parameters of the model.
- The resolution and number of pressure levels must match the data. Lower resolution and fewer levels will run a bit faster. Data resolution only affects the encoder/decoder.
- All our models predict precipitation. However, ERA5 includes precipitation, while HRES does not. Our models marked as "ERA5" take precipitation as input and expect ERA5 data as input, while model marked "ERA5-HRES" do not take precipitation as input and are specifically trained to take HRES-fc0 as input (see the data section below).

We provide three pre-trained models.
1. `GraphCast`, the high-resolution model used in the GraphCast paper (0.25 degree resolution, 37 pressure levels), trained on ERA5 data from 1979 to 2017,

2. `GraphCast_small`, a smaller, low-resolution version of GraphCast (1 degree resolution, 13 pressure levels, and a smaller mesh), trained on ERA5 data from 1979 to 2015, useful to run a model with lower memory and compute constraints,

3. `GraphCast_operational`, a high-resolution model (0.25 degree resolution, 13 pressure levels) pre-trained on ERA5 data from 1979 to 2017 and fine-tuned on HRES data from 2016 to 2021. This model can be initialized from HRES data (does not require precipitation inputs).


In [ ]:
# @title Choose the model

params_file_options = [
    name for blob in gcs_bucket.list_blobs(prefix=dir_prefix+"params/")
    if (name := blob.name.removeprefix(dir_prefix+"params/"))]  # Drop empty string.

random_mesh_size = widgets.IntSlider(
    value=4, min=4, max=6, description="Mesh size:")
random_gnn_msg_steps = widgets.IntSlider(
    value=4, min=1, max=32, description="GNN message steps:")
random_latent_size = widgets.Dropdown(
    options=[int(2**i) for i in range(4, 10)], value=32,description="Latent size:")
random_levels = widgets.Dropdown(
    options=[13, 37], value=13, description="Pressure levels:")


params_file = widgets.Dropdown(
    options=params_file_options,
    description="Params file:",
    layout={"width": "max-content"})

source_tab = widgets.Tab([
    widgets.VBox([
        random_mesh_size,
        random_gnn_msg_steps,
        random_latent_size,
        random_levels,
    ]),
    params_file,
])
source_tab.set_title(0, "Random")
source_tab.set_title(1, "Checkpoint")
widgets.VBox([
    source_tab,
    widgets.Label(value="Run the next cell to load the model. Rerunning this cell clears your selection.")
])


In [ ]:
# @title Load the model

source = source_tab.get_title(source_tab.selected_index)

if source == "Random":
  params = None  # Filled in below
  state = {}
  model_config = graphcast.ModelConfig(
      resolution=0,
      mesh_size=random_mesh_size.value,
      latent_size=random_latent_size.value,
      gnn_msg_steps=random_gnn_msg_steps.value,
      hidden_layers=1,
      radius_query_fraction_edge_length=0.6)
  task_config = graphcast.TaskConfig(
      input_variables=graphcast.TASK.input_variables,
      target_variables=graphcast.TASK.target_variables,
      forcing_variables=graphcast.TASK.forcing_variables,
      pressure_levels=graphcast.PRESSURE_LEVELS[random_levels.value],
      input_duration=graphcast.TASK.input_duration,
  )
else:
  assert source == "Checkpoint"
  with gcs_bucket.blob(f"{dir_prefix}params/{params_file.value}").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)
  params = ckpt.params
  state = {}

  model_config = ckpt.model_config
  task_config = ckpt.task_config
  print("Model description:\n", ckpt.description, "\n")
  print("Model license:\n", ckpt.license, "\n")

print(model_config)
print(task_config)

## Load the example data

Several example datasets are available, varying across a few axes:
- **Source**: fake, era5, hres
- **Resolution**: 0.25deg, 1deg, 6deg
- **Levels**: 13, 37
- **Steps**: How many timesteps are included

Not all combinations are available.
- Higher resolution is only available for fewer steps due to the memory requirements of loading them.
- HRES is only available in 0.25 deg, with 13 pressure levels.

The data resolution must match the model that is loaded.

Some transformations were done from the base datasets:
- We accumulated precipitation over 6 hours instead of the default 1 hour.
- For HRES data, each time step corresponds to the HRES forecast at leadtime 0, essentially providing an "initialisation" from HRES. See HRES-fc0 in the GraphCast paper for further description. Note that a 6h accumulation of precipitation is not available from HRES, so our model taking HRES inputs does not depend on precipitation. However, because our models predict precipitation, we include the ERA5 precipitation in the example data so it can serve as an illustrative example of ground truth.
- We include ERA5 `toa_incident_solar_radiation` in the data. Our model uses the radiation at -6h, 0h and +6h as a forcing term for each 1-step prediction. If the radiation is missing from the data (e.g. in an operational setting), it will be computed using a custom implementation that produces values similar to those in ERA5.

In [ ]:
# @title Get and filter the list of available example datasets

dataset_file_options = [
    name for blob in gcs_bucket.list_blobs(prefix=dir_prefix+"dataset/")
    if (name := blob.name.removeprefix(dir_prefix+"dataset/"))]  # Drop empty string.

def data_valid_for_model(
    file_name: str, model_config: graphcast.ModelConfig, task_config: graphcast.TaskConfig):
  file_parts = parse_file_parts(file_name.removesuffix(".nc"))
  return (
      model_config.resolution in (0, float(file_parts["res"])) and
      len(task_config.pressure_levels) == int(file_parts["levels"]) and
      (
          ("total_precipitation_6hr" in task_config.input_variables and
           file_parts["source"] in ("era5", "fake")) or
          ("total_precipitation_6hr" not in task_config.input_variables and
           file_parts["source"] in ("hres", "fake"))
      )
  )


dataset_file = widgets.Dropdown(
    options=[
        (", ".join([f"{k}: {v}" for k, v in parse_file_parts(option.removesuffix(".nc")).items()]), option)
        for option in dataset_file_options
        if data_valid_for_model(option, model_config, task_config)
    ],
    description="Dataset file:",
    layout={"width": "max-content"})
widgets.VBox([
    dataset_file,
    widgets.Label(value="Run the next cell to load the dataset. Rerunning this cell clears your selection and refilters the datasets that match your model.")
])

In [ ]:
# @title Load weather data

if not data_valid_for_model(dataset_file.value, model_config, task_config):
  raise ValueError(
      "Invalid dataset file, rerun the cell above and choose a valid dataset file.")

with gcs_bucket.blob(f"{dir_prefix}dataset/{dataset_file.value}").open("rb") as f:
  example_batch = xarray.load_dataset(f).compute()

assert example_batch.dims["time"] >= 3  # 2 for input, >=1 for targets

print(", ".join([f"{k}: {v}" for k, v in parse_file_parts(dataset_file.value.removesuffix(".nc")).items()]))

example_batch

In [ ]:
# @title Choose training and eval data to extract
train_steps = widgets.IntSlider(
    value=1, min=1, max=example_batch.sizes["time"]-2, description="Train steps")
eval_steps = widgets.IntSlider(
    value=example_batch.sizes["time"]-2, min=1, max=example_batch.sizes["time"]-2, description="Eval steps")

widgets.VBox([
    train_steps,
    eval_steps,
    widgets.Label(value="Run the next cell to extract the data. Rerunning this cell clears your selection.")
])

In [ ]:
# @title Extract training and eval data

train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
    **dataclasses.asdict(task_config))

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{eval_steps.value*6}h"),
    **dataclasses.asdict(task_config))

print("All Examples:  ", example_batch.dims.mapping)
print("Train Inputs:  ", train_inputs.dims.mapping)
print("Train Targets: ", train_targets.dims.mapping)
print("Train Forcings:", train_forcings.dims.mapping)
print("Eval Inputs:   ", eval_inputs.dims.mapping)
print("Eval Targets:  ", eval_targets.dims.mapping)
print("Eval Forcings: ", eval_forcings.dims.mapping)


In [ ]:
# @title Load normalization data

with gcs_bucket.blob(dir_prefix+"stats/diffs_stddev_by_level.nc").open("rb") as f:
  diffs_stddev_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob(dir_prefix+"stats/mean_by_level.nc").open("rb") as f:
  mean_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob(dir_prefix+"stats/stddev_by_level.nc").open("rb") as f:
  stddev_by_level = xarray.load_dataset(f).compute()

In [ ]:
# @title Build functions to run model

def construct_wrapped_graphcast(model_config, task_config, return_graphs=False):

    core_predictor = graphcast.GraphCast(model_config, task_config)
    # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
    # from/to float32 to/from BFloat16.
    predictor = casting.Bfloat16Cast(core_predictor)
    # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
    # BFloat16 happens after applying normalization to the inputs/targets.
    predictor = normalization.InputsAndResiduals(
        predictor,
        diffs_stddev_by_level=diffs_stddev_by_level,
        mean_by_level=mean_by_level,
        stddev_by_level=stddev_by_level
    )
    # Wraps everything so the one-step model can produce trajectories.
    predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)

    if return_graphs:
        return predictor, core_predictor  # Return both
    else:
        return predictor

@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):

    predictor, core_predictor = construct_wrapped_graphcast(
        model_config, task_config, return_graphs=True
    )

    prediction = predictor(inputs, targets_template=targets_template, forcings=forcings)

    # Extract graphs from the raw GraphCast instance
    graphs = {
        "grid2mesh_graph": core_predictor._grid2mesh_graph_structure,
        "mesh_graph": core_predictor._mesh_graph_structure,
        "mesh2grid_graph": core_predictor._mesh2grid_graph_structure,
    }

    return prediction, graphs

def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

def with_params(fn):
  return functools.partial(fn, params=params, state=state)

def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

# Run the model

Note that the cell below may take a while (possibly minutes) to run the first time you execute them, because this will include the time it takes for the code to compile. The second time running will be significantly faster.

This use the python loop to iterate over prediction steps, where the 1-step prediction is jitted. This has lower memory requirements than the training steps below, and should enable making prediction with the small GraphCast model on 1 deg resolution data for 4 steps.

In [ ]:
# @title Initialise model

params, state = run_forward.init(
    jax.random.PRNGKey(0),
    model_config=model_config,
    task_config=task_config,
    inputs=train_inputs,
    targets_template=train_targets,
    forcings=train_forcings
)

run_forward_apply = drop_state(with_params((with_configs(
    run_forward.apply))))


In [ ]:
output, graphs = run_forward_apply(
    rng=jax.random.PRNGKey(1),
    inputs=train_inputs,
    targets_template=train_targets,
    forcings=train_forcings
)

In [ ]:
# @title Save graph data

# Upload it to Google Drive
drive.mount('/content/drive')

name = "split=4_random_init"

# Save the graphs as shown above to the drive
# grid2mesh
filename = "/grid2mesh_typedgraph"+name+".pkl"

# Save to a path in your Drive
with open("/content/drive/MyDrive"+filename, "wb") as f:
    pickle.dump(graphs["grid2mesh_graph"], f)

# mesh graph
filename = "/mesh_typedgraph"+details+".pkl"

with open("/content/drive/MyDrive/mesh_typedgraph"+details+".pkl", "wb") as f:
    pickle.dump(graphs["mesh_graph"], f)

# mesh2grid
filename = "/mesh2grid_typedgraph.pkl"

with open("/content/drive/MyDrive/mesh2grid_typedgraph"+details+".pkl", "wb") as f:
    pickle.dump(graphs["mesh2grid_graph"], f)

In [ ]:
# @title Import graphs

# drive.mount('/content/drive')
# # Define the path to the file
# filename = "/content/drive/MyDrive/mesh_typedgraph.pkl"

# # Open the file and load the graph
# with open(filename, "rb") as f:
#     mesh_graph = pickle.load(f)

In [ ]:
# @title Look at saved structural data

graphs["grid2mesh_graph"].nodes

In [ ]:
graphs["grid2mesh_graph"].nodes["grid_nodes"].n_node # number of grid nodes

In [ ]:
graphs["grid2mesh_graph"].nodes["grid_nodes"].features.shape

In [ ]:
graphs["grid2mesh_graph"].nodes["grid_nodes"].features[:15] # features of first 3 grid nodes.
# Features are: [cos(latitute_in_radians), cos(longitude in radians), sin(longitude in radians)]
# so one can know latitude and longitude of node.

In [ ]:
graphs["grid2mesh_graph"].edge_by_name("grid2mesh").n_edge # number of edges

In [ ]:
graphs["grid2mesh_graph"].edge_by_name("grid2mesh").indices.senders.shape

In [ ]:
graphs["grid2mesh_graph"].edge_by_name("grid2mesh").indices.senders[:15]

In [ ]:
graphs["grid2mesh_graph"].edge_by_name("grid2mesh").indices.receivers.shape

In [ ]:
graphs["grid2mesh_graph"].edge_by_name("grid2mesh").indices.receivers[:15]

In [ ]:
graphs["grid2mesh_graph"].edge_by_name("grid2mesh").features.shape

In [ ]:
graphs["grid2mesh_graph"].edge_by_name("grid2mesh").features[:10]
# The features are: [ distance, dx, dy, dz ]
# These are computed in a receiver-local coordinate system — meaning the origin is the mesh node (receiver),
# and the edge vector points from mesh node to grid node (i.e., backward along the edge).

In [ ]:
# @title Plot structures

# Load the relevant node sets and edges
graph = graphs["grid2mesh_graph"]
grid_nodes = graph.nodes["grid_nodes"].features
mesh_nodes = graph.nodes["mesh_nodes"].features
edge_set = graph.edge_by_name("grid2mesh")
senders = edge_set.indices.senders
receivers = edge_set.indices.receivers
edge_features = edge_set.features

In [ ]:
# Use only the first [] (sender) grid nodes
selected_grid_indices = np.arange(1)
print(selected_grid_indices)

In [ ]:
# Find all edges from those grid nodes
mask = np.isin(senders, selected_grid_indices)
sub_senders = senders[mask]
sub_receivers = receivers[mask]
sub_edge_features = edge_features[mask]
print(sub_senders)
print(sub_receivers)
print(sub_edge_features)

In [ ]:
# Unique mesh nodes connected to selected grid nodes
unique_mesh_indices = np.unique(sub_receivers)
print(unique_mesh_indices)

In [ ]:
# Convert Cartesian back to lat/lon for grid and mesh nodes
def cartesian_to_latlon(feat1, feat2, feat3):
    lat = np.degrees(np.arcsin(feat1))  # latitude
    lon = np.degrees(np.arctan2(feat3, feat2))  # longitude from x and y
    lon = (lon + 360) % 360  # wrap around
    return lat, lon

print(cartesian_to_latlon(1, -1, 0))
print(cartesian_to_latlon(1, 0, 1))
print(cartesian_to_latlon(-1, 0, -1))

In [ ]:
# Create NetworkX graph
G = nx.DiGraph()

# Add grid nodes (blue)
pos = {}
for i in selected_grid_indices:
    feat1, feat2, feat3 = grid_nodes[i]
    lat, lon = cartesian_to_latlon(feat1, feat2, feat3)
    label = f"S{i}\n({lat:.1f},{lon:.1f})"
    G.add_node(f"S{i}", pos=(lon, lat), color='lightblue', label=label)
    pos[f"S{i}"] = (lon, lat)


# Add mesh nodes (orange)
for i in unique_mesh_indices:
    feat1, feat2, feat3 = mesh_nodes[i]
    lat, lon = cartesian_to_latlon(feat1, feat2, feat3)
    label = f"R{i}\n({lat:.1f},{lon:.1f})"
    G.add_node(f"R{i}", pos=(lon, lat), color='orange', label=label)
    pos[f"R{i}"] = (lon, lat)

# Add edges with direction and weights from feature[0] (distance)
for s, r, feat in zip(sub_senders, sub_receivers, sub_edge_features):
    G.add_edge(f"S{s}", f"R{r}", weight=feat[0])


# Draw the graph
plt.figure(figsize=(12, 8))
node_colors = [data['color'] for _, data in G.nodes(data=True)]
labels = nx.get_node_attributes(G, 'label')
nx.draw(G, pos, with_labels=False, node_color=node_colors, edge_color='gray', node_size=800)
nx.draw_networkx_labels(G, pos, labels, font_size=8)

plt.title("Grid2Mesh Graph (First ... Grid Nodes and Connected Mesh Nodes)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()


In [ ]:
# @title Visualise mesh graph

# plot ever ... node
freq = 1

# Helper: Convert Cartesian to lat/lon
def cartesian_to_latlon(x, y, z):
    lat = np.degrees(np.arcsin(x))  # Assuming x is up (vertical)
    lon = np.degrees(np.arctan2(z, y))  # From y and z
    lon = (lon + 360) % 360
    return lat, lon

# Extract data
mesh_graph = graphs["mesh_graph"]
mesh_nodes = mesh_graph.nodes["mesh_nodes"].features
edge_set = mesh_graph.edge_by_name("mesh").indices
senders = edge_set.senders
receivers = edge_set.receivers
edge_features = mesh_graph.edge_by_name("mesh").features

selected_mesh_indices = np.arange(0, mesh_nodes.shape[0], freq)
mask = np.isin(senders, selected_mesh_indices) | np.isin(receivers, selected_mesh_indices) # want all sending and receiving edges associated with selected nodes
sub_senders = senders[mask]
sub_receivers = receivers[mask]

# Convert to lat/lon
latlons = {} # dict where e.g. index 100 has lotlon coords: (lat,lon)
for i in selected_mesh_indices:
    x, y, z = mesh_nodes[i]
    lat, lon = cartesian_to_latlon(x, y, z)
    latlons[i] = (lat, lon)

# Edge traces
edge_traces = []
for s, r in zip(sub_senders, sub_receivers):
    if s in latlons and r in latlons:   # ensure both s and r in nodes selected
        lat_s, lon_s = latlons[s]
        lat_r, lon_r = latlons[r]
        edge_traces.append(go.Scattergeo(
            lon=[lon_s, lon_r],
            lat=[lat_s, lat_r],
            mode='lines',
            line=dict(width=0.5, color='gray'),
            opacity=0.5,
            showlegend=False
        ))

# Node trace
lats = [latlons[i][0] for i in latlons]
lons = [latlons[i][1] for i in latlons]

node_trace = go.Scattergeo(
    lon=lons,
    lat=lats,
    mode='markers+text',
    marker=dict(size=4, color='orange'),
    textposition="top center",
    name='Mesh Nodes'
)

# Final plot
fig = go.Figure(data=[node_trace] + edge_traces)
fig.update_geos(
    projection_type="equirectangular",
    showland=True, landcolor="rgb(243, 243, 243)",
    showcountries=True,
    lataxis_showgrid=True, lonaxis_showgrid=True,
)
fig.update_layout(
    title="Mesh Graph on Globe",
    height=800,
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=False
)
fig.show()
plt.savefig("mesh_graph.png")